<a href="https://colab.research.google.com/github/Shailesh0209/x_tools_in_ds_dipoma-iitm/blob/main/X_W7_Design_your_output.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Design_your_output: Tools for specialized visualizations: network of actors

In [ ]:
import numpy as np
import pandas as pd